<a href="https://colab.research.google.com/github/kevinyang372/Prolog_Expert_System/blob/master/LBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt install swi-prolog

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  autoconf automake autopoint autotools-dev debhelper dh-autoreconf
  dh-strip-nondeterminism file gettext gettext-base intltool-debian
  libarchive-cpio-perl libarchive-zip-perl libbsd-dev libedit-dev
  libfile-stripnondeterminism-perl libgmp-dev libgmpxx4ldbl libmagic-mgc
  libmagic1 libmail-sendmail-perl libncursesw5-dev libossp-uuid16 libsigsegv2
  libsys-hostname-long-perl libtimedate-perl libtool m4 po-debconf
  swi-prolog-nox swi-prolog-x
Suggested packages:
  autoconf-archive gnu-standards autoconf-doc dh-make dwz gettext-doc
  libasprintf-dev libgettextpo-dev gmp-doc libgmp10-doc libmpfr-dev
  ncurses-doc uuid libtool-doc gcj-jdk m4-doc libmail-box-perl elpa-ediprolog
The following NEW package

In [2]:
!pip install pyswip

In [0]:
import pandas as pd
import numpy as np

In [0]:
# Loading the data and renaming the columns 
# The names now correspond to the statement types
 
data = pd.read_csv('/content/CS152 LBA data - Sheet1.csv', skiprows=1)
col_names = ['name', 'hasCuisine', 'priceRange', 'mealTypes', 'distance',
             'isVegetarian', 'glutenFree', 'hasLanguage', 'transportation']
data.columns = col_names

In [0]:
data.head(5)

### Building the KB

In [0]:
# Converted Euro signs to three price levels: low, medium, high in Excel 

# all_commands = []
listCommands = []

for k in range(len(data)):
  row = data.loc[k, :]
  resName = row[0]
  # concatenate words in the name, converted to lowercase (can fix to join with lower dash later)
  resName = ''.join(e for e in resName.lower() if e.isalnum()) 
  # listCommands = [] # all commands per single restaurant name
  for i in range(1, len(row)):
    category = data.columns[i]
    items = row[i]
    if type(items) == str:
      items = row[i].split(', ')
      for item in items:
        item = ''.join(e for e in item.lower() if e.isalnum())
        command = "{0}({1}, {2})".format(category, resName, item)
        listCommands.append(command)
    else:
        command = "{0}({1}, {2})".format(category, resName, items)
        listCommands.append(command)
  # all_commands.append(listCommands) 

In [8]:
# Checking to see how Middle eastern got converted

print(listCommands[12:14])

['hasCuisine(maroush, lebanese)', 'hasCuisine(maroush, middleeastern)']


In [0]:
from pyswip import Prolog
prolog = Prolog()

# Assert KB inputs for all commands
for command in listCommands:
  prolog.assertz(command)

### Queries

In [0]:
# Includes all the variables apart from Language

prolog.assertz("query(X, Cuisine, Price, Distance, Vegetarian, Mealtype, Glutenfree, Language, Transport) :- hasCuisine(X, Cuisine), priceRange(X, Price), distance(X, Y), Y < Distance, isVegetarian(X, Vegetarian), glutenFree(X, Glutenfree), mealTypes(X, Mealtype), hasLanguage(X, Language), transportation(X, Transport)")

In [0]:
# Latest update: changed distance parameter default value to 1000 from -1

# search wrapper breaks down the attributes that have multiple values
def searchWrapper(**attributes):

  # initialize dictionary for single and multiple arguments
  d_single, d_multi = {}, {}
  to_query = []

  for k, v in attributes.items():
    if ',' in v:
      # if there are multiple arguments record them in a query
      d_multi[k] = v.split(',')
      to_query.append(k)
    else:
      d_single[k] = v

  # use backtracking to find all combinations
  def runAllCombination(query):
    if not query: return [{}] # base case: return empty dictionary

    p = query[0]
    res = runAllCombination(query[1:])
    fin = []

    # update new dictionaries with all possible combinations
    for arg in d_multi[p]:
      for pos in res:
        temp = dict(pos)
        temp.update({p: arg})
        fin.append(temp)

    return fin

  result = set()

  # run through all possible combinations and update result
  for m in runAllCombination(to_query):
    m.update(d_single)
    result.update(search_new(**m))

  return result

def search_new(cuisine = '_', price = '_', distance = '1000', isvegetarian = '_', mealtype = '_', isglutenfree = '_', language = '_' , transport = '_'):
  to_query = "query(X, %s)" % (",".join([cuisine, price, distance, isvegetarian, mealtype, isglutenfree, language, transport]))
  try:
    return set([soln["X"] for soln in prolog.query(to_query)])
  except:
    return set()

In [34]:
searchWrapper(cuisine = 'lebanese, middleeastern, thai, italian')

{'lapiadina', 'lemongrass', 'maroush', 'paglia'}

### Test cases

In [36]:
# Search case 1: All restaurants within a 15 min walking distance

searchWrapper(distance = "15")

{'burgermeisterkottbussertor',
 'hasirburger',
 'lapiadina',
 'longmarchcanteen',
 'maroush',
 'milchzucker',
 'mundvoll',
 'paglia',
 'quepasamexicana',
 'santamaria',
 'sushiforyou'}

In [37]:
# Search case 2: A restaurant within a 10 min walking distance in a low price range
# that has English-speaking staff, vegetarian and dinner options

searchWrapper(distance = "10", price="low", isvegetarian="y", mealtype="dinner", language = 'english', transport="walk")

{'lapiadina', 'maroush'}

In [38]:
# Search case 3: Filter by cuisine (italian) and price (medium)

searchWrapper(cuisine = 'italian', price="medium")

{'paglia'}

In [39]:
# Search case 4: Group dinner search; prefer American cuisine in a low price range and near the residence

searchWrapper(cuisine = 'american', distance = "15", price="low")

{'burgermeisterkottbussertor'}

In [40]:
from ipywidgets import interact
interact(searchWrapper, cuisine = "_", price = "_", distance = "1000", isvegetarian = "_", mealtype = '_', isglutenfree = '_', transport = '_')
# interact(search, cuisine = "_", price = "_", distance = "-1")

interactive(children=(Text(value='_', description='cuisine'), Text(value='_', description='price'), Text(value…

<function __main__.searchWrapper>